<a href="https://colab.research.google.com/github/ehdghks316/itwill_studyGroup_D/blob/main/4_13%ED%8C%8C%EC%9D%B4%EC%8D%AC(train_test_split%2C%ED%98%BC%EB%8F%99%ED%96%89%EB%A0%AC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from collections import Counter

survey = pd.read_csv('c:/data/naive_survey.csv',header=None, names=['sentens','class']) #컬럼이름이 없으면 header=None하고 names로 이름 설정
survey

x_train,x_test,y_train,y_test = train_test_split(survey['sentens'],survey['class'],test_size=0.2) # 학습데이터와 테스트 데이터로 나누기
len(x_train)
len(x_test)
Counter(y_train)
Counter(y_test)

# 학습데이터
cv = CountVectorizer(ngram_range=(2,2))
x_train = cv.fit_transform(x_train)
cv.get_feature_names()
x_train.toarray()

# 테스트데이터
x_test = cv.transform(x_test) # 위에서 fit작업이 완료되어 사전이 만들어져 있고 transform 작업만하면 됨
x_test.toarray()

# 학습 모델
nb = MultinomialNB()
nb.fit(x_train,y_train)

# 예측
y_predict = nb.predict(x_test)
y_predict == y_test # 정답 일치 확인
sum(y_predict == y_test) # 정답이 몇개인지 확인
sum(y_predict == y_test)/3 # 정답률

# 정답률 함수
accuracy_score(y_test,y_predict) # accuracy_score(테스트데이터 정답라벨, 예측한 테스트데이터)


x_train,x_test,y_train,y_test = train_test_split(survey['sentens'],survey['class'],test_size=0.2) # 학습데이터와 테스트 데이터로 나누기
Counter(y_train)
Counter(y_test)

#한꺼번에 모든문장을 넣어서 형태소분석을 하는건 메모리에 문제 생길 수 있으니 
#한 문장씩 형태소 분석하는 것 (찾아보기 이전에 수업에서 했었음)
from konlpy.tag import Okt
okt = Okt()

def okt_pos(arg):
    token_corpus = []
    for i in okt.pos(arg):
        if i[1] in ['Noun','Adjective']:
            token_corpus.append(i[0])
    token_corpus = [word for word in token_corpus if len(word) >= 2]
    return token_corpus

okt_pos(x_train[0])

# 학습데이터
cv = CountVectorizer(tokenizer=okt_pos)
x_train = cv.fit_transform(x_train)
cv.get_feature_names()
x_train.toarray()

# 테스트데이터
x_test = cv.transform(x_test) # 위에서 fit작업이 완료되어 사전이 만들어져 있고 transform 작업만하면 됨
x_test.toarray()

# 학습 모델
nb = MultinomialNB()
nb.fit(x_train,y_train)

# 예측
y_predict = nb.predict(x_test)
y_predict == y_test # 정답 일치 확인
sum(y_predict == y_test) # 정답이 몇개인지 확인
sum(y_predict == y_test)/3 # 정답률

# 정답률 함수
accuracy_score(y_test,y_predict)


★★ 혼동행렬(confusion matrix)
- 모델 성능을 평가할 때 사용되는 지표
- 예측값이 실제값을 얼마나 정확하게 예측했는지를 보여주는 행렬

# 혼동행렬 crosstab
pd.crosstab(y_test,y_predict)


        predict(예측)
   col_0  긍정  부정
   class        
실 긍정      1   0
제 부정      1   1

            예측(긍정) 예측(부정)
------------------------
실제(긍정)      TP          FN
실제(부정)      FP          TN

TP(True Positive) : 참긍정, 긍정이라고 예측을 했는데 실제도 긍정
TN(True Negative) : 참부정, 부정이라고 예측을 했는데 실제도 부정
FP(False Positive) : 거짓긍정, 긍정이라고 예측을 했는데 실제는 부정
FN(False Negative) : 거짓부정, 부정이라고 예측을 했는데 실제는 긍정 (제일 줄여야하는 부분 ex) 암이 아니라고 했는데 알고보니 암이다.)

from sklearn.metrics import confusion_matrix,classification_report

confusion_matrix(y_test,y_predict)
array([[1, 0],
       [1, 1]], dtype=int64)

print(classification_report(y_test,y_predict)) # 포맷문자가 섞여있어서 print해줘야 깔끔
              precision    recall  f1-score   support

          긍정       0.50      1.00      0.67         1
          부정       1.00      0.50      0.67         2

    accuracy                           0.67         3
   macro avg       0.75      0.75      0.67         3
weighted avg       0.83      0.67      0.67         3

▶정확도(accuracy)
- 모델이 입력된 데이터에 대해 얼마나 정확하게 예측하는 지를 나타내는 지표

정확도 = 예측결과와 실제값이 동일한 건수 / 전체 데이터 수
        = (TP+TN) / (TP+TN+FP+FN)

▶정밀도(precision) # 예측에서 실제
- 정밀도는 긍정 클래스에 속한다고 예측한 값이 실제도 긍정클래스에 속하는 비율
- 정밀도는 부정 클래스에 속한다고 예측한 값이 실제도 부정클래스에 속하는 비율

긍정정밀도 = TP / (TP+FP)
부정정밀도 = FN / (TN+FN)

▶재현율(recall) # 실제에서 예측
- 실제값 중에서 모델이 검출한 실제값의 비율을 나타내는 지표

긍정재현율 = TP / (TP+FN)
부정재현율 = TN / (FP+TN)

▶f1-scor
- 정밀도도 중요하고 재현율도 중요한데 둘 중 무엇을 사용해야 할지 고민될 때
두 값을 조화평균 내서 하나의 수치로나타낸 지표
# like 시간당 속도로 시속구하는 것 처럼 두 값을 조화평균내서 수치로

긍정 f1-score = (긍정재현율*긍정정밀도*2) / (긍정재현율+긍정정밀도) 
부정 f1-score = (부정재현율*부정정밀도*2) / (부정재현율+부정정밀도) 